In [2]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create the agents.
# model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
from autogen_core.models import ModelFamily
model_client = OpenAIChatCompletionClient(
    # model="qwen2.5:7b",
    model="llama3.2:3b",
    base_url="http://localhost:11434/v1",
    api_key="placeholder",
    model_info={
        "vision": False,
        "function_calling": True,
        "json_output": False,
        "family": ModelFamily.ANY,
    },
)

assistant = AssistantAgent("assistant", model_client=model_client)
user_proxy = UserProxyAgent("user_proxy", input_func=input)  # Use input() to get user input from console.

# Create the termination condition which will end the conversation when the user says "APPROVE".
termination = TextMentionTermination("APPROVE")

# Create the team.
team = RoundRobinGroupChat([assistant, user_proxy], termination_condition=termination)

# Run the conversation and stream to the console.
stream = team.run_stream(task="Write a 4-line poem about the ocean.")
# Use asyncio.run(...) when running in a script.
await Console(stream)

---------- user ----------
Write a 4-line poem about the ocean.
---------- assistant ----------
Here's a 4-line poem about the ocean:

Deep blue waves crash on the shore,
A soothing melody forever more.
The ocean's vastness touches the sky,
A sight to cherish, as the tide goes by.
---------- user_proxy ----------
APPROVE


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a 4-line poem about the ocean.', type='TextMessage'), TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=59, completion_tokens=46), metadata={}, content="Here's a 4-line poem about the ocean:\n\nDeep blue waves crash on the shore,\nA soothing melody forever more.\nThe ocean's vastness touches the sky,\nA sight to cherish, as the tide goes by.", type='TextMessage'), UserInputRequestedEvent(source='user_proxy', models_usage=None, metadata={}, request_id='2f742cc6-2528-455e-9acb-a88f59ac9f8f', content='', type='UserInputRequestedEvent'), TextMessage(source='user_proxy', models_usage=None, metadata={}, content='APPROVE', type='TextMessage')], stop_reason="Text 'APPROVE' mentioned")

In [6]:
# Once the team stops, the turn count will be reset. When you resume the team, it will start from 0 again. However, the team’s internal state will be preserved, for example, the RoundRobinGroupChat will resume from the next agent in the list with the same conversation history.
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create the agents.
# model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
from autogen_core.models import ModelFamily
model_client = OpenAIChatCompletionClient(
    # model="qwen2.5:7b",
    model="llama3.2:3b",
    base_url="http://localhost:11434/v1",
    api_key="placeholder",
    model_info={
        "vision": False,
        "function_calling": True,
        "json_output": False,
        "family": ModelFamily.ANY,
    },
)

assistant = AssistantAgent("assistant", model_client=model_client)

# Create the team setting a maximum number of turns to 1.
team = RoundRobinGroupChat([assistant], max_turns=1)

task = "Write a 4-line poem about the ocean."
while True:
    # Run the conversation and stream to the console.
    stream = team.run_stream(task=task)
    # Use asyncio.run(...) when running in a script.
    await Console(stream, output_stats=True)
    # Get the user response.
    task = input("Enter your feedback (type 'exit' to leave): ")
    if task.lower().strip() == "exit":
        break

---------- user ----------
Write a 4-line poem about the ocean.
---------- assistant ----------
Waves crash on the sandy shore,
The ocean's roar, forever more.
Deep blue waters, endless seas,
A mystery that's full of glee.

 Terminated.
[Prompt tokens: 59, Completion tokens: 36]
---------- Summary ----------
Number of messages: 2
Finish reason: Maximum number of turns 1 reached.
Total prompt tokens: 59
Total completion tokens: 36
Duration: 0.97 seconds
---------- user ----------
Can you make it about a person and its relationship with the ocean
---------- assistant ----------
Here is a 4-line poem:

Tides of solitude, I find my home
Where waves caress the soul, I am not alone
In the depths below, I see a truth revealed
A connection that within me forever dwells.

Terminated.
[Prompt tokens: 117, Completion tokens: 54]
---------- Summary ----------
Number of messages: 2
Finish reason: Maximum number of turns 1 reached.
Total prompt tokens: 117
Total completion tokens: 54
Duration: 1.49 

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import Handoff
from autogen_agentchat.conditions import HandoffTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create an OpenAI model client.
# model_client = OpenAIChatCompletionClient(
#     model="gpt-4o",
#     # api_key="sk-...", # Optional if you have an OPENAI_API_KEY env variable set.
# )
from autogen_core.models import ModelFamily
model_client = OpenAIChatCompletionClient(
    model="qwen2.5:7b",
    # model="llama3.2:3b",
    base_url="http://localhost:11434/v1",
    api_key="placeholder",
    model_info={
        "vision": False,
        "function_calling": True,
        "json_output": False,
        "family": ModelFamily.ANY,
    },
)

# Create a lazy assistant agent that always hands off to the user.
lazy_agent = AssistantAgent(
    "lazy_assistant",
    model_client=model_client,
    handoffs=[Handoff(target="user", message="Transfer to user.")],
    # system_message="If you cannot complete the task, transfer to user. Otherwise, when finished, respond with 'TERMINATE'.",
    system_message="If you cannot complete the task or tool is not available, transfer to user. Otherwise, when finished with good data, respond with 'TERMINATE'.",
)

# Define a termination condition that checks for handoff messages.
handoff_termination = HandoffTermination(target="user")
# Define a termination condition that checks for a specific text mention.
text_termination = TextMentionTermination("TERMINATE")

# Create a single-agent team with the lazy assistant and both termination conditions.
lazy_agent_team = RoundRobinGroupChat([lazy_agent], termination_condition=handoff_termination | text_termination)

# Run the team and stream to the console.
task = "What is the weather in New York today?"
await Console(lazy_agent_team.run_stream(task=task), output_stats=True)

---------- user ----------
What is the weather in New York today?


---------- lazy_assistant ----------
[FunctionCall(id='call_u2rtvpx1', arguments='{}', name='transfer_to_user')]
[Prompt tokens: 167, Completion tokens: 84]
---------- lazy_assistant ----------
[FunctionExecutionResult(content='Transfer to user.', name='transfer_to_user', call_id='call_u2rtvpx1', is_error=False)]
---------- lazy_assistant ----------
Transfer to user.
---------- Summary ----------
Number of messages: 4
Finish reason: Handoff to user from lazy_assistant detected.
Total prompt tokens: 167
Total completion tokens: 84
Duration: 5.30 seconds


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='What is the weather in New York today?', type='TextMessage'), ToolCallRequestEvent(source='lazy_assistant', models_usage=RequestUsage(prompt_tokens=167, completion_tokens=84), metadata={}, content=[FunctionCall(id='call_u2rtvpx1', arguments='{}', name='transfer_to_user')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='lazy_assistant', models_usage=None, metadata={}, content=[FunctionExecutionResult(content='Transfer to user.', name='transfer_to_user', call_id='call_u2rtvpx1', is_error=False)], type='ToolCallExecutionEvent'), HandoffMessage(source='lazy_assistant', models_usage=None, metadata={}, target='user', content='Transfer to user.', context=[], type='HandoffMessage')], stop_reason='Handoff to user from lazy_assistant detected.')

In [13]:
await Console(lazy_agent_team.run_stream(task="The weather in New York is sunny."))

---------- user ----------
The weather in New York is sunny.
---------- lazy_assistant ----------
TERMINATE


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='The weather in New York is sunny.', type='TextMessage'), TextMessage(source='lazy_assistant', models_usage=RequestUsage(prompt_tokens=218, completion_tokens=4), metadata={}, content='TERMINATE', type='TextMessage')], stop_reason="Text 'TERMINATE' mentioned")